In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
import math
import matplotlib.pyplot as plt
from tools.stat_tools.stat_tools import *
from tools.visualization.visualization import *
import matplotlib.dates as mdates
import sklearn.metrics as sk
import glob, os  

pd.options.mode.chained_assignment = None 

In [ ]:
path = r'C:\Users\Isabel Cristina\Desktop\Algo_trading_py\data'
df_csv = pd.concat(map(pd.read_csv, glob.glob(os.path.join(path, "darwinex_demo*.csv"))))
df_csv = df_csv[df_csv['Profit'].notna()]
df_csv['Magic Number'] = df_csv['Magic Number'].astype(int).astype(str)

df_csv.columns = [c.replace(' ', '_') for c in df_csv.columns]
df_csv['Open_Date'] = pd.to_datetime(df_csv['Open_Date'])
df_csv['Close_Date'] = pd.to_datetime(df_csv['Close_Date'])

# Selecting sets that have at least one operation in te last month
last_month_date = dt.date.today() - pd.DateOffset(months=1)
magic_last_month = df_csv[df_csv.Close_Date >= last_month_date]['Magic_Number'].unique()
df_csv = df_csv[df_csv['Magic_Number'].isin(magic_last_month)]

# Deleting sets that doesn't get minimun trades and days in real
df_res = df_csv.groupby(['Magic_Number'], as_index =  False)['Profit'].count()
df_res = df_res.join(df_csv.groupby(['Magic_Number'])['Open_Date'].min(), on = ['Magic_Number'])
df_res['Today_Date'] = pd.to_datetime('today', format='%Y-%m-%d')
df_res['Days_in_Real'] = (df_res['Today_Date']-df_res['Open_Date']).dt.days
df_res = df_res[df_res.Profit > 20]

# Selecting magic_numbers and cleaning final csv
magic_to_test = df_res.Magic_Number.unique()
df_csv = df_csv[df_csv['Magic_Number'].isin(magic_to_test)]

In [ ]:
# Creating all info needed
# Instancia
df_ins = stat_tools(df_csv)

# DFs with de info
df_stats = df_ins.get_stat()
df_aux = df_stats.copy()

# Deleting sets that are loosing 
df_stats = df_stats[df_stats.Profit > 0]

# Deleting sets with PF lower than 1.3
df_stats = df_stats[df_stats.PF >= 1.3]

# Deleting sets with r2 lower than 0.7
df_stats = df_stats[df_stats.R2_KPI >= 0.7]

# Deleting sets with SQN less than 1
df_stats = df_stats[df_stats.SQN > 1]

# Deleting sets with Win/DD_Ratio less than 1
df_stats = df_stats[df_stats['Win/DD_Ratio'] > 1]

# Deleting sets with Trades/Month less than 1.5
df_stats = df_stats[df_stats['Trades/Month'] > 1.5]

In [ ]:
magic_good = df_stats.index.unique()
df_csv = df_csv[df_csv['Magic_Number'].isin(magic_good)]
df_csv = df_csv.reset_index()

In [ ]:
df_ins_good = stat_tools(df_csv)
df_resumen = df_ins_good.profit_per_period() 
df_magic_evolution = df_ins_good.set_magic_evolution()
df_port_evolution = df_ins_good.set_port_evolution()

In [ ]:
fig_ins = sets_visualization(df_csv, magic_good)

fig = fig_ins.evolution_plot(df_port_evolution, df_magic_evolution, df_resumen)

fig.show()

In [ ]:
fig_2 = fig_ins.month_profit_heatmap(df_resumen)
fig_2.show()

In [ ]:
print("--- Total sets: {}".format(len(df_stats)))
pd.set_option('display.max_rows', 999)
df_stats

In [ ]:
fig = fig_ins.plot_regression()
fig.show()